# Libraries

In [61]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from keras_preprocessing.sequence import pad_sequences

import numpy as np
import os

# Preprocessing

In [62]:
# base path
directory_path = './labels'
# current directory
c_dir = os.getcwd()

# all actions
actions = np.array(sorted([folder for folder in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, folder))])) # sorted to follow folder arrangement

# specific actions
# actions = np.array(['alligator', 'flower', 'kiss', 'listen', 'orange'])
# actions = np.array(['afternoon', 'house', 'again', 'open', 'kiss', 'sorry'])

In [63]:
# create a dictionary for int representation of actions
label_map = {label:num for num, label in enumerate(actions)}
label_map

{'afternoon': 0,
 'again': 1,
 'alligator': 2,
 'base': 3,
 'door': 4,
 'flower': 5,
 'hello': 6,
 'house': 7,
 'how': 8,
 'kiss': 9,
 'listen': 10,
 'open': 11,
 'orange': 12,
 'see': 13,
 'sorry': 14,
 'why': 15}

Note that at this point, we will not access the video folder, only the numpy folder.

In [64]:
sequences, labels = [], []  # sequence -> video, labels -> action
for action in actions:
    no_actions = len(os.listdir(os.path.join(c_dir, 'labels', action)))
    print('Opening path:', os.path.join(c_dir, 'labels', action))
    print(f'Number of instances: {no_actions}')
    for num in range(1, no_actions + 1):
        window = []         # window -> single frame
        file = str(action) + "_" + str(num)
        no_frames_per_action = len(os.listdir(os.path.join(c_dir, 'labels', action, file)))
        print(f'Number of frames in {file}: {no_frames_per_action}')
        for frame_num in range(1, no_frames_per_action + 1):
            res = np.load(os.path.join(c_dir, 'labels', action, file,  "{}.npy".format(frame_num)))     # res -> coordinate key points
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])
    print('-'*75)

Opening path: /mnt/d/GitHub/SSLrecognition/train_data/labels/afternoon
Number of instances: 40
Number of frames in afternoon_1: 31
Number of frames in afternoon_2: 30
Number of frames in afternoon_3: 30
Number of frames in afternoon_4: 30
Number of frames in afternoon_5: 31
Number of frames in afternoon_6: 31
Number of frames in afternoon_7: 31
Number of frames in afternoon_8: 30
Number of frames in afternoon_9: 31
Number of frames in afternoon_10: 31
Number of frames in afternoon_11: 31
Number of frames in afternoon_12: 31
Number of frames in afternoon_13: 31
Number of frames in afternoon_14: 31
Number of frames in afternoon_15: 31
Number of frames in afternoon_16: 31
Number of frames in afternoon_17: 31
Number of frames in afternoon_18: 31
Number of frames in afternoon_19: 31
Number of frames in afternoon_20: 31
Number of frames in afternoon_21: 31
Number of frames in afternoon_22: 31
Number of frames in afternoon_23: 31
Number of frames in afternoon_24: 31
Number of frames in aftern

In [65]:
# due to difference in number of frames, pad x and y
x = np.array(pad_sequences(sequences, dtype = 'float', padding = 'post', value = 0))
y = to_categorical(labels).astype(int)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, stratify = y)

In [66]:
input_shape = (x_train.shape[1], x_train.shape[2])
print(x_train.shape)
print(x_test.shape)

(392, 267, 225)
(98, 267, 225)


In [67]:
y_test

array([[0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [68]:
(2/3)*(x_train.shape[2]+y_train.shape[1])

160.66666666666666

# Models

## LSTM

In [69]:
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.callbacks import TensorBoard, TerminateOnNaN, EarlyStopping

from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [70]:
# for logging of data with TensorBoard
log_dir = os.path.join(c_dir, 'Logs')
tb_callback = TensorBoard(log_dir = log_dir)

# to end training when failure happens ie. loss == nan
term = TerminateOnNaN()

# to stop training early if there is no change in loss
early = EarlyStopping(monitor = 'loss', patience = 5)

In [71]:
def choose_lstm(n):
    if n == 1:
        model = Sequential()
        model.add(LSTM(64, return_sequences = True, input_shape = input_shape))
        model.add(Dropout(0.2))
        model.add(Bidirectional(LSTM(64)))
        model.add(Dropout(0.2))
        model.add(Dense(actions.shape[0], activation = "softmax"))

        return model

    elif n == 2:
        model = Sequential()
        model.add(LSTM(64, return_sequences = True, input_shape = (117, 225)))
        model.add(LSTM(128, return_sequences = True))
        model.add(LSTM(64, return_sequences = False))
        model.add(Dense(64))
        model.add(Dense(32))
        model.add(Dense(8))
        model.add(Dense(actions.shape[0], activation = "softmax"))

        return model

    elif n == 3:
        model = Sequential()
        model.add(LSTM(128, return_sequences = True, input_shape = (117, 225)))
        model.add(Dropout(0.1))
        model.add(LSTM(64, return_sequences = False))
        model.add(Dropout(0.1))
        model.add(Dense(actions.shape[0], activation = "softmax"))

        return model
    
    elif n == 4:
        model = Sequential()
        model.add(LSTM(128, return_sequences=True, activation='relu', input_shape = input_shape))
        model.add(Dropout(0.2))
        model.add(LSTM(256, return_sequences=True, activation='relu'))
        model.add(Dropout(0.2))
        model.add(LSTM(256, return_sequences=False, activation='relu'))
        # model.add(BatchNormalization())
        model.add(Dense(256, activation='relu'))
        model.add(Dense(128, activation='relu'))
        model.add(Dense(64, activation='relu'))
        model.add(Dense(actions.shape[0], activation='softmax'))

        return model

In [72]:
model = choose_lstm(1)
opt = keras.optimizers.Adam(learning_rate = 0.001)
model.compile(optimizer = opt, loss = "categorical_crossentropy", metrics = ['categorical_accuracy'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_8 (LSTM)               (None, 267, 64)           74240     
                                                                 
 dropout_8 (Dropout)         (None, 267, 64)           0         
                                                                 
 bidirectional_4 (Bidirecti  (None, 128)               66048     
 onal)                                                           
                                                                 
 dropout_9 (Dropout)         (None, 128)               0         
                                                                 
 dense_4 (Dense)             (None, 16)                2064      
                                                                 
Total params: 142352 (556.06 KB)
Trainable params: 142352 (556.06 KB)
Non-trainable params: 0 (0.00 Byte)
______________

In [73]:
model.fit(x_train, y_train, epochs = 100, batch_size = 16, validation_split = 0.2, callbacks = [term, tb_callback])

Epoch 1/100
20/20 [==============================] - 4s 77ms/step - loss: 2.5615 - categorical_accuracy: 0.1661 - val_loss: 2.3756 - val_categorical_accuracy: 0.1772
Epoch 2/100
20/20 [==============================] - 1s 45ms/step - loss: 2.0489 - categorical_accuracy: 0.3227 - val_loss: 2.0213 - val_categorical_accuracy: 0.2911
Epoch 3/100
20/20 [==============================] - 1s 42ms/step - loss: 1.6795 - categorical_accuracy: 0.4505 - val_loss: 1.8264 - val_categorical_accuracy: 0.3165
Epoch 4/100
20/20 [==============================] - 1s 42ms/step - loss: 1.5816 - categorical_accuracy: 0.4089 - val_loss: 1.7712 - val_categorical_accuracy: 0.2532
Epoch 5/100
20/20 [==============================] - 1s 44ms/step - loss: 1.4747 - categorical_accuracy: 0.4313 - val_loss: 1.5681 - val_categorical_accuracy: 0.4177
Epoch 6/100
20/20 [==============================] - 1s 41ms/step - loss: 1.3150 - categorical_accuracy: 0.5272 - val_loss: 1.5406 - val_categorical_accuracy: 0.4177
Epoc

In [74]:
# take model predictions
res = model.predict(x_test)

4/4 [==============================] - 1s 15ms/step


In [75]:
res

array([[1.03312668e-04, 6.00084313e-05, 7.47255835e-05, ...,
        9.79686081e-01, 1.18928151e-02, 8.76464648e-04],
       [1.90223844e-04, 7.80099246e-04, 5.05305035e-03, ...,
        4.80425078e-04, 1.12958229e-03, 8.77167471e-03],
       [8.76470658e-05, 1.46894227e-03, 3.27843875e-02, ...,
        4.55667236e-04, 1.49985781e-05, 7.53068423e-04],
       ...,
       [6.68247580e-04, 2.42819326e-04, 8.16543251e-02, ...,
        7.84843578e-04, 1.03556877e-03, 8.46534022e-05],
       [2.66178162e-04, 9.69008077e-04, 1.06978533e-03, ...,
        1.40674214e-03, 7.94374355e-05, 1.61573663e-03],
       [5.80676933e-05, 1.00301746e-04, 2.04387167e-03, ...,
        7.10007007e-05, 1.05716765e-03, 1.59568866e-04]], dtype=float32)

In [76]:
y_true = np.argmax(y_test, axis = 1).tolist()
y_pred = np.argmax(res, axis = 1).tolist()

In [77]:
print(y_true)
print(y_pred)

[13, 5, 10, 2, 9, 10, 5, 14, 6, 15, 2, 2, 9, 5, 6, 9, 10, 2, 0, 5, 10, 2, 0, 8, 9, 15, 2, 8, 14, 12, 8, 3, 10, 12, 3, 5, 8, 3, 6, 12, 9, 2, 12, 13, 13, 6, 7, 9, 14, 12, 5, 14, 2, 0, 10, 0, 15, 8, 10, 13, 12, 10, 13, 10, 13, 6, 5, 8, 2, 6, 8, 12, 9, 12, 0, 4, 11, 9, 5, 15, 13, 3, 10, 0, 12, 13, 0, 6, 8, 1, 12, 5, 9, 0, 2, 9, 6, 5]
[13, 5, 10, 5, 9, 10, 2, 14, 6, 15, 2, 10, 9, 5, 6, 9, 9, 2, 0, 5, 10, 2, 0, 8, 9, 15, 2, 8, 14, 5, 8, 3, 10, 5, 3, 12, 8, 3, 6, 5, 15, 2, 8, 13, 13, 6, 7, 2, 14, 5, 5, 14, 2, 0, 2, 0, 15, 8, 10, 13, 12, 10, 13, 2, 13, 6, 5, 8, 2, 6, 8, 12, 5, 12, 8, 11, 7, 0, 5, 15, 13, 3, 5, 8, 2, 13, 0, 6, 8, 1, 5, 5, 9, 0, 10, 9, 6, 9]


In [78]:
print(f'Accuracy: {round(accuracy_score(y_true, y_pred)*100, 1)}%')

Accuracy: 74.5%


In [79]:
if accuracy_score(y_true, y_pred) >= 0.8:
    model.save('test_model_70.keras')
else:
    print('Model accuracy insufficient')

# keras.models.load_model("test_model.keras")

Model accuracy insufficient


In [80]:
# import os
# import shutil

# folder_path = './averaged_np_labels/'
# parent_files = os.listdir(os.path.join(folder_path))
# write_path = './labels/'

# for parent_file in parent_files:
#     parent_path = os.path.join(folder_path, parent_file)
#     export_path = os.path.join(write_path, parent_file)
#     os.mkdir(export_path)
#     print(f'Created new directory: {export_path}')
#     for i in range(1, len(os.listdir(os.path.join(folder_path, parent_file)))+1):
#         new_subfolder = f'{parent_file}_{i}'
#         os.mkdir(os.path.join(export_path, new_subfolder))
#         print(f'Created new subdirectory: {new_subfolder}')

#         source = os.path.join(parent_path, f'video{i}')
#         destination = os.path.join(os.path.join(export_path, new_subfolder))

#         sourcefolder = os.listdir(os.path.join(parent_path, f'video{i}'))
#         for file in sourcefolder:
#             file_to_copy = os.path.join(os.path.join(parent_path, f'video{i}'), file)
#             shutil.copy(file_to_copy, destination)